In [ ]:
from src.arctic import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Given Data
- from Institute of Atmospheric Physics Kühlungsborn (IAP)
- Differentiated between ERA5 and UA-ICON(NWP) data

1. ERA5: daily means at pressure levels and analysis
    - Intercomparison with Hannachi et al. (2011)
    - SWE1.tar: contains major warming diagnostics
    - SWE1.tar: elliptic diagnostics of Z10
    - SWX1.tar: extended diagnostics (reachable via VPN)
    - SWC1.tar: classfiication synthesis (reachable via VPN)
2. NWP4A60: data at pressure levels and analysis
    - SWM2.tar: major warming diagnostics
    - SWE2.tar: elliptic diagnostics of Z10
    - SWX2.tar: extended diagnostics (VPN)
    - SWC2.tar: classification synthesis (VPN)

#### File structure
netCDF files are 3-dimensional data, investigated with ncdump. The .tar-files contain all output from Christoph Zülickes diagnosis, in particular:
- SWM: ASCII values in 2 files, 
    1. *cen.csv with the major warming events as identified with their central day when the wind turned from westerly to easterly along with some characteristics and
    2. *msw.csv with daily lists with wind and a marker mw for major warming
- SWE: 1 csv file
    - *d.csv: daily list of indices, including the aspect ratio of the mother vortex (ar), centrail latitude (latcent) and area (area)

In [ ]:
# load data
# preliminary, ideally include api from climate data storage (cds)

# Skips all lines starting with C
# Reads only lines starting with D
def read_data(input_file):
    with open(input_file, "r", encoding="utf-8") as f:
        filtered_lines = [line for line in f if line.startswith("D")]
    
    # Convert filtered lines into DataFrame
    from io import StringIO
    df = pd.read_csv(
        StringIO("".join(filtered_lines)),
        delimiter=",",
        low_memory=False
    )
    return df

era5_d = read_data("data/SWXClust/ERA5/e5e1_d.csv")
era5_cen = read_data("data/SWXClust/ERA5/e5m1t-cen.csv")
era5_msw = read_data("data/SWXClust/ERA5/e5m1t-msw.csv")

nwp_d = read_data("data/SWXClust/NWP4A60/i4a60e2_d.csv")
nwp_cen= read_data("data/SWXClust/NWP4A60/i4a60m2t-cen.csv")
nwp_msw = read_data("data/SWXClust/NWP4A60/i4a60m2t-msw.csv")

In [ ]:
# delete space in front of strings
def no_white_space(df, sep = ' '):
    col = []
    for c in df.columns:
        col.append(c.split(sep)[-1])
    df.columns = col
    
# convert strings to actual dates
def to_date(df, col, format='mixed'):
    df[col] = pd.to_datetime(df[col], format=format)

# remove white space from column names
no_white_space(era5_d)
no_white_space(era5_cen)
no_white_space(era5_msw)

# no_white_space(nwp_d)
# no_white_space(nwp_cen)
# no_white_space(nwp_msw)


# remove empty rows from files
era5_d = era5_d[era5_d['string'].notna()]
era5_cen = era5_cen[era5_cen['string'].notna()]
era5_msw = era5_msw[era5_msw['string'].notna()]

# nwp_d = nwp_d[nwp_d['string'].notna()]
# nwp_cen = nwp_cen[nwp_cen['string'].notna()]
# nwp_msw = nwp_msw[nwp_msw['string'].notna()]


# change string to datetime for better comparison
to_date(era5_d, 'string')
to_date(era5_cen, 'string')
to_date(era5_msw, 'string')

# drop where string is nan (empty lines)
era5_d = era5_d.replace('                   ' ,np.NaN)

era5_d.head()

## Features in data and their description
#### for *cen files

35x9

| ERA5            | D_cen                       | num                   | counter_cen                                            | time_cen                 | string                  | pers                 | Umax                      | Imax                     | Iacc                            |
|-----------------|-----------------------------|-----------------------|--------------------------------------------------------|--------------------------|-------------------------|----------------------|---------------------------|--------------------------|---------------------------------|
| Values          | 'D'                         | [1,35]                | [26, 21189]                                            | [2438057.25, 2459220.35] | [1963-01-27,2021-05-01] | [2.0,29]             | [-26.9,-0.9]              | [1.7,780.1]              | [1.4,316.5]                     |
| Explanation     | Line belongs to data in csv | Counter of SSW events | Central time counter <br/> index matching measurements | Julian central date      | human readable date     | persistence of event | max. east wind during SSW | max. SSW event intensity | accumulated SSW event intensity |
| Condition       | -                           | -                     |                                                        |                          |                         |                      |                           |                          |                                 |
| Normalization   | no                          | no                    |                                                        |                          |                         |                      |                           |                          |                                 |
| Units           | -                           | -                     | run step                                               | run time                 | run time                | days                 | m/s                       | days m/s                 | days m/s                        |
| Value for model | None                        | Probably none         | Merge key                                              | Merge key                | Merge key               |                      |                           |                          |                                 |

#### for *msw files

21914x6

| ERA5            | D_msw          | counter_msw                                                 | time_msw                 | string                  | u             | MW                  |
|-----------------|----------------|-------------------------------------------------------------|--------------------------|-------------------------|---------------|---------------------|
| Values          | 'D'            | [1, 21914]                                                  | [2438032.25, 2459944.25] | String encoding of date | [-28.8, 73.5] | [0,1] (i.e. no/yes) |
| Explanation     | Line with data | Counter of events, <br/>each daily measurement gets counter | Julian central date      | human readable date     | zonal wind    | marks major warming |
| Condition       | -              |                                                             |                          |                         | ??            |                     |
| Normalization   | no             | no                                                          |                          | no                      |               | no, encode as 0-1   |
| Value for Model | None           | Merge key                                                   | Merge key                | Merge key               | include       | should be included  |



#### for *d files

22403x35

21425 + 978 (D vs. S)

**bold font**: unclear meaning

| ERA5      | Values                   | Explanation                                                                                                  | Condition    | Normalisation  | Value for Model                                                                                           |
|-----------|--------------------------|--------------------------------------------------------------------------------------------------------------|--------------|----------------|-----------------------------------------------------------------------------------------------------------|
| D         | 'D'                      | Line with data                                                                                               |              | -              |                                                                                                           |
| counter   | [1,21914]                | counter of daily measurements                                                                                | -            | -              |                                                                                                           |
| time_d    | [2438031.25, 2459944.25] | Julian day                                                                                                   | -            |                | Merge Key                                                                                                 |
| string    | [1963-01-01, 2022-12-30] | Human readable/string form of date                                                                           |              |                | Merge Key                                                                                                 |
| year      | [1963                    | numeric form of year                                                                                         |              |                | useful                                                                                                    |
| month     | [1,12]                   | numeric form of month                                                                                        |              |                | useful                                                                                                    |
| day       | [1,31]                   | numeric form of day                                                                                          |              |                | useful                                                                                                    |
| hour      | 18                       | numeric form of hour                                                                                         |              |                | None, daily measurement                                                                                   |
| level     | 10                       | Pressure level in hPa, ca. 30km                                                                              |              |                | Currently not huge as not varied,  important for comparison with other studies                            |
| **edge**  | [28.9,        32.3]      | Edge of Vortex? °N? Assumption as circle/ellipsoid around pole?<br/>Strong correlation with generated labels |              |                |                                                                                                           |
| form     | [' D', ' S']             | Form of Vortex: <br/>Displaced (4599 in era5_hannachi), <br/> Split (502 in era5_hannachi)                   |              | Encode as int? | Important, evaluation of clusters                                                                         |
| **area**  | [7105100.0,162460000.0]  | Area of Vortex [km^2]                                                                                        |              |                |                                                                                                           |
| **obj_area** | [11163.0,  3850800.0]    | Objective area of vortex [km^2], diference to area?                                                          |              |                |                                                                                                           |
| latcent   | [50.823, 89.966]         | Latitude of centre                                                                                           |              |                |                                                                                                           |
| loncent   | [-89.993,269.95]         | Longitude of centre                                                                                          |              |                |                                                                                                           |
| theta     | [-1.5707, 1.5797]        | angle of major axis                                                                                          |              |                |                                                                                                           |
| ar        | [1.0001, 4.8253]         | Aspect ratio of polar vortex                                                                                 |              |                |                                                                                                           |
| kurtosis  | [-0.81021,10.235]        | Kurtosis of polar vortex                                                                                     |              |                |                                                                                                           |
| area1     | [na, 29034864.000]       | Area of split vortex 1 [km^2]                                                                                | Split event? |                | Important, but more for evaluation <br/> may irritate cluster algorithms (implicit information of events) |
| obj_area1 | [na, 592939.812]         |                                                                                                              |              |                |                                                                                                           |
| latcent1  | [na, 88.211]             | Latitude of centroid 1                                                                                       |              |                |                                                                                                           |
| loncent1  | [na, 269.912]            | Longitude of centroid 1                                                                                      |              |                |                                                                                                           |
| theta1    | [na, -1.569]             |                                                                                                              |              |                |                                                                                                           |
| ar1       | [na, 3.651]              | Aspect ratio                                                                                                 |              |                |                                                                                                           |
| area2     | [na, 33281544.000]       | Area of split vortex 2 [km^2]                                                                                |              |                |                                                                                                           |
| obj_area2 | [na, 635659.312]         |                                                                                                              |              |                |                                                                                                           |
| latcent2  | [na, 87.856]             |                                                                                                              |              |                |                                                                                                           |
| loncent2  | [na, 202.300]            |                                                                                                              |              |                |                                                                                                           |
| theta2    | [na, -1.570]             |                                                                                                              |              |                |                                                                                                           |
| ar2       | [na, 4.117]              |                                                                                                              |              |                |                                                                                                           |
| **amp0**  | [27.6, 32.3]             | zonal mean                                                                                                   |              |                |                                                                                                           |
| **amp1**  | [0.000024,  0.0489]      | amplitude of wave 1                                                                                          |              |                |                                                                                                           |
| **amp2**  | [0.000003, 0.0543]       | amplitude of wave 2                                                                                          |              |                |                                                                                                           |
| **lon1**  | [-180.0, 180.0]          | angle of wave 1                                                                                              |              |                |                                                                                                           |
| **lon2**  | [-180.0, 180.0]          | angle of wave 2                                                                                              |              |                |                                                                                                           |

### Focus and To Do
- [x] Understand relation between data sets, merge via 'string' (i.e. concrete date) possible
- [x] ERA5 Data, replicate results from Hannachi
- [ ] descriptive statistics:
    - [ ] PSA/SSA
    - [x] correlation matrix/chi2
- [ ] hierarchical Clustering, k-means, k-nearest neighbours


In [ ]:
# merge era5 data to one big dataframe
era5 = era5_d.merge(era5_msw, on ='string', how='left', suffixes=[None, '_msw']).merge(era5_cen, on='string', how='left', suffixes = ['_d', '_cen'])
era5.columns

## Reconstruction of same conditions as Hannachi et al.
- time: winterperiod (december to march) from 1958 to 2002, activity peaks in dec - jan, oscilliation period around 120-130d
    - **Note: only data after 1963 available 
- 7 stratospheeric variables:

| temp. (lat1)     | temp (lat2)      | temp. (lat3)     | avg. wind (lat1)      | avg. wind (lat2) | wave-1                    | wave-2                    |
|------------------|------------------|------------------|-----------------------|------------------|---------------------------|---------------------------|
| Zonally averaged | Zonally averaged | Zonally averaged | Zonally averaged <br/ | Zonally averaged | Normalised by zonal winds | Normalised by zonal winds |
| 60-70°N          | 70-80°N          | 80-90°N          | 60-70°N               | 70-80°N          | 60°N                      | 60°N                      |

- 7 pressure levels between 100 to 1hPa
- geometrical features: latitude centroid (lat), area (area), aspect ratio (AR) - kurtosis omitted due to domination of outliers

#### Approaches
1. Hierarchical Clustering
    - Used distance: Euclidean distance
    - Distance between Cluster: Complete Linkage to avoid chain and spheroidal tendencies of single and average linkage
    - Gap statistic to find number of clusters with Poisson point process (standard)
    - SSA to deal with seasonality and its variation

|               | between points in Cluster | Different                                  |
|---------------|---------------------------|--------------------------------------------|
| Closest       | Single Linkage            | Centroid Linkage                           |
| Most Distance | **Complete Linkage**      | Average Linkage (Average of all distances) |

In [ ]:
# correlation within data
plot.plot_correlation(era5, savefig="output/corr.png")

    ## Jupyter Notebook to demonstrate usage of ARTIC
### Open questions
- Connectivity matrix from Hannachi et al.? Do they have one? With what created?
- Different amount of samples between *_d and *_msw
- Differences between era40 data and era5 data

#### To Do
- [ ] Check wie Cluster mit Daten/Split events zusammen passen
- [ ] Check Area (big correlation with labels) and check what filtered area in Hannachi et al. is

### Handling NaN Values
- *cen-file produces most NaN values in data due to merch and marking special events &rArr; 0 not used in *num, counter, time, pers, Imax, Iacc*,**Umax** has values only below -0.9 

In [ ]:
# reproduce same circumstances as hannachi et al.
# time winterperiod (december to march) 1958 - 2002 
var = ['year', 'month','day', # removed string so everything is a numeric datetype
       'edge', 'theta',
       'area', 'latcent', 'loncent', 'ar',  # geometric variables (latcent, area, ar, excess kurtosis) from Hannachi et al., level = 10 for all measurements -> droped
       'amp0', 'amp1', 'amp2', 'lon1', 'lon2', # unclear importance, included in this version
       'MW', 'u', # variables from *msw file
       'pers', 'Umax', 'Imax', 'Iacc', # variables form *cen file
       'form'] # Discuss! Usage of form!

# enforce same time constraints
era5_hannachi = era5[(era5['string'] < '01-01-2003') & 
                     (era5['month']).isin([12,1,2,3])][var]

# handling NaN values and string encodings
era5_hannachi['MW'] = pd.to_numeric(era5_hannachi.MW.replace({'   ': 0, ' MW' : 1}))
era5_hannachi[era5_cen.columns[-4:]] = era5_hannachi[era5_cen.columns[-4:]].fillna(0) 
# check for completeness
era5_hannachi.info()

##### Assumption
- Data are ready to use and preprocessed

In [ ]:
era5_hannachi['form'] = era5_hannachi['form'].replace({' D' : 0, ' S': 1})
#var[:-1] # ohne D_d und form

In [ ]:
from sklearn.cluster import AgglomerativeClustering
X = era5_hannachi[var[:-1]]
hc = AgglomerativeClustering(n_clusters=3,
                             linkage='complete',
                             compute_distances=True).fit(X)
hc_label = hc.labels_

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, n_init='auto', random_state=42).fit(X)
kmeans_label = kmeans.labels_

In [ ]:
era5_hannachi['hc'] = hc_label
era5_hannachi['kmeans'] = kmeans_label
# compare label with mw, c/d, s/d
corr = era5_hannachi.corr()
corr[['kmeans', 'hc']].style.background_gradient(vmin=-1, vmax=1, cmap='RdBu')

In [ ]:
print(era5_hannachi.shape)
print("Label distribution by hierarchical clustering: ")
print('0 : ', era5_hannachi[era5_hannachi.hc == 0].shape) # 74
print('1 : ', era5_hannachi[era5_hannachi.hc == 1].shape) # 403
print('2 : ', era5_hannachi[era5_hannachi.hc == 2].shape) # 4624

print("Label distribution by kmeans clustering: ")
print('0 : ', era5_hannachi[era5_hannachi.kmeans == 0].shape) # 74
print('1 : ', era5_hannachi[era5_hannachi.kmeans == 1].shape) # 403
print('2 : ', era5_hannachi[era5_hannachi.kmeans == 2].shape) # 4624

### Results from Hannachi et al.

| AR and Latc of vortex/AR, Lat, filtered vortex area | D  | U  | S  | Total |
|-----------------------------------------------------|----|----|----|-------|
| D                                                   | 7  |  3 | 0  | 10    |
| U                                                   | 4  | 80 | 2  | 86    |
| S                                                   | 1  | 0  | 3  | 4     |
| Total                                               | 12 | 83 | 5  | 100   |

### Results with Agglomorative Clustering:

| Label | Absolut | Perc (%) | Comment                                                                                                      |
|-------|---------|----------|--------------------------------------------------------------------------------------------------------------|
| 0     | 74      | 1,5      | Probably S<br/>Remains the same when removing other features<br/>74 when only area, latcent and ar were used |
| 1     | 403     | 8        | Probably D  <br/> Varies wildly when features removed                                                        |
| 2     | 4624    | 90,6     | Probably U  <br/>Varies wildly when features removed                                                         |
| Total | 5101    | 100      | --                                                                                                           |

Influence of different linkages:
- complete: used by Hannachi, dendogram does not indicate 3 clusters
- ward: appears similar to kmeans in distribution
- single: makes very weird Cluster with distribution 70, 4, 5027; indicates 3 Clusters
- average: makes also weird Cluster with distribution 70, 23, 5008; indicates 3 Clusters

### Results with kmeans Clustering:

| Label | Absolut | Perc (%)   | Comment                                                                                                      |
|-------|---------|------------|--------------------------------------------------------------------------------------------------------------|
| 0     | 2000    | 39,25      | Maybe D  <br/> Varies wildly, depending on features and random seed                                          |
| 1     | 79      | 1,5        | Probably S<br/>Remains the same when removing other features<br/>74 when only area, latcent and ar were used |
| 2     | 3022    | 59,25      | Maybe U  <br/>Varies wildly, just as D                                                                       |
| Total | 5101    | 100        | --                                                                                                           

#### Evaluation
S and U are underrepresented. Different measures?
Go for troubleshooting:
- Outliers? (~75?)
- Can I - based on my available data - differentiate between S, U, D?
- Data differences (s. previous part)
- Description of found cluster matching with Hannachi/general what are they?

Hierarchical cluster depends severly on **area**

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.title('Dendogram of Agglomerative Clustering')
plot.plot_dendrogram(hc, truncate_mode='level', p=3, savefig='output/dendrogram_hc.png')
plt.show()

In [ ]:
pc=13
compute_pca(era5_hannachi, comp=pc, plot_type='3D', savefig='output/pca_era5_3D.png')
compute_pca(era5_hannachi, comp=pc, plot_type='2D', savefig='output/pca_era5_2D.png')
era5_hannachi['label'] = era5_hannachi['hc']
compute_pca(era5_hannachi, comp=pc, plot_type='3D', savefig='output/pca_era5_hc_3D.png')
compute_pca(era5_hannachi, comp=pc, plot_type='2D', savefig='output/pca_era5_hc_2D.png')
era5_hannachi['label'] = era5_hannachi['kmeans']
compute_pca(era5_hannachi, comp=pc, plot_type='3D', savefig='output/pca_era5_kmeans_3D.png')
compute_pca(era5_hannachi, comp=pc, plot_type='2D', savefig='output/pca_era5_kmeans_2D.png')

In [3]:
def own_agg(x):
    return x.quantile(0.75)

plot_radar(era5_hannachi, label='hc')

NameError: name 'plot_radar' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd

def plot_with_dataframe(df, P=10, T=-50):
    """
    Plots a polar stereographic map using data from a DataFrame and local coastlines.

    Parameters:
    - df (pd.DataFrame): DataFrame with latitude, longitude, and other relevant columns.
    - P pressure to compute geopotential height
    - T assumed temperature at that height
    """
    
    # Generate sample geopotential height data (replace with real computations)
    lon = np.linspace(0, 360, 361)
    lat = np.linspace(90, 20, 71)  # From the North Pole to mid-latitudes
    lon2d, lat2d = np.meshgrid(lon, lat)
    
    # Note: gph kann sich ändern, als Linien ergänzen?
    # # compute geopotential height
    T = T+ZERO_DEG # convert T to Kelvin
    z = R*T/g*np.log(P0/P)
    gph = np.full_like(lon2d, z) 

    # Extract specific columns from the DataFrame
    lat_points = df['latcent']
    lon_points = df['loncent']

    # Create a polar stereographic plot
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': ccrs.NorthPolarStereo()})

    # Plot data as filled contours, show changing gph as shades of grey
    # include color bar if so
    # contour = ax.contourf(lon2d, lat2d, gph, levels=20, transform=ccrs.PlateCarree(), cmap='Greys')

    # Add contour lines for gph
    ax.contour(lon2d, lat2d, gph, levels=10, colors='black', linewidths=1, transform=ccrs.PlateCarree())

    # Add local coastlines from Cartopy's features
    coastlines = cfeature.NaturalEarthFeature('physical', 'coastline', '50m', edgecolor='black', facecolor='none')
    ax.add_feature(coastlines, linewidth=0.5)

    # Add points from the DataFrame
    ax.scatter(lon_points, lat_points, color='red', s=10, transform=ccrs.PlateCarree(), label='Data Points')

    # Add gridlines
    ax.gridlines(draw_labels=False, color='gray', linestyle='--')

    # Set the extent (view) for the polar plot
    # ax.set_extent([-180, 180, 20, 90], crs=ccrs.PlateCarree())

    # Add a colorbar
    # cbar = plt.colorbar(contour, ax=ax, orientation='horizontal', pad=0.05, shrink=0.8)
    # cbar.set_label('Geopotential Height (gpm)')

    # Add a legend
    ax.legend(loc='lower left')
    fig.autofmt_xdate(rotation=45)


# Call the function
plot_with_dataframe(era5_hannachi[era5_hannachi['year'] == 1963][['latcent', 'loncent']])


In [ ]:
# preprocessing

In [ ]:
# clustering

In [1]:
# visualisation

In [2]:
era5[era5['year'] == 1963][['latcent', 'loncent']]

NameError: name 'era5' is not defined